Creation of a .msh file from a .stl file using gmsh. The first step is to reconstruct the parametric model form the stl file and them mesh it. With that regard, I beleive it is better to start from an stp file and not from a stl file.

Generate mesh from stp file using gmsh. The problem with this following code being that the tesselation size is not controlled. 

In [1]:
import gmsh
import sys

# ========== USER SETTINGS ==========
step_file = "Living_hinge_1.step"    # Your STEP file
mesh_file = "Living_hinge_1.msh"
target_element_size = 0.01     # Max element size
visualize = True
# ===================================

gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.add("MeshFromSTEP")

# Load STEP file
gmsh.option.setString("Geometry.OCCTargetUnit", "M")  # or "MM" depending on model units
gmsh.model.occ.importShapes(step_file)
gmsh.model.occ.synchronize()

# Disable curvature-based sizing and others that may override global size
gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 0)
gmsh.option.setNumber("Mesh.MeshSizeFromPoints", 0)
gmsh.option.setNumber("Mesh.MeshSizeExtendFromBoundary", 0)

# Apply mesh size to all points explicitly
points = gmsh.model.getEntities(dim=0)
for p in points:
    gmsh.model.mesh.setSize([p], target_element_size)

# Add physical groups (optional but recommended for solvers)
surfaces = gmsh.model.getEntities(dim=2)
volumes = gmsh.model.getEntities(dim=3)

if surfaces:
    surface_tags = [s[1] for s in surfaces]
    gmsh.model.addPhysicalGroup(2, surface_tags)
    gmsh.model.setPhysicalName(2, 1, "walls")

if volumes:
    volume_tags = [v[1] for v in volumes]
    gmsh.model.addPhysicalGroup(3, volume_tags)
    gmsh.model.setPhysicalName(3, 2, "domain")

# Generate 3D mesh
gmsh.model.mesh.generate(3)

# Save mesh
gmsh.write(mesh_file)

# GUI view
if visualize:
    gmsh.fltk.run()

gmsh.finalize()

Info    :  - Label 'Shapes/Living_hinge_1 v1' (3D)
Info    :  - Color (0.627451, 0.627451, 0.627451) (3D & Surfaces)
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Circle)
Info    : [ 20%] Meshing curve 4 (Line)
Info    : [ 20%] Meshing curve 5 (Circle)
Info    : [ 20%] Meshing curve 6 (Line)
Info    : [ 30%] Meshing curve 7 (Circle)
Info    : [ 30%] Meshing curve 8 (Line)
Info    : [ 30%] Meshing curve 9 (Line)
Info    : [ 40%] Meshing curve 10 (Line)
Info    : [ 40%] Meshing curve 11 (Line)
Info    : [ 40%] Meshing curve 12 (Line)
Info    : [ 50%] Meshing curve 13 (Circle)
Info    : [ 50%] Meshing curve 14 (Circle)
Info    : [ 50%] Meshing curve 15 (Line)
Info    : [ 60%] Meshing curve 16 (Line)
Info    : [ 60%] Meshing curve 17 (Circle)
Info    : [ 60%] Meshing curve 18 (Line)
Info    : [ 70%] Meshing curve 19 (Line)
Info    : [ 70%] Meshing curve 20 (Circle)
Info    : [ 70%] Meshing curve 2

Control tesselation size using gmsh. 

In [2]:
import gmsh
import sys
import numpy as np

# Initialize Gmsh
gmsh.initialize()

# Load the STEP file (replace 'input.stp' with your STEP file path)
gmsh.model.occ.importShapes("Living_hinge_1.step")

# Synchronize the geometry to update the model
gmsh.model.occ.synchronize()

# Get all volumes in the model
volumes = gmsh.model.getEntities(dim=3)

# Define physical groups for all volumes (optional, for solver compatibility)
for i, vol in enumerate(volumes):
    gmsh.model.addPhysicalGroup(vol[0], [vol[1]], name=f"volume_{i+1}")

# Define a distance-based mesh size field
# Mesh size will be smaller near the origin (0,0,0) and larger farther away
field_id = 1
gmsh.model.mesh.field.add("Distance", field_id)
gmsh.model.mesh.field.setNumbers(field_id, "PointsList", [1])  # Reference point at origin
gmsh.model.mesh.field.setNumber(field_id, "Sampling", 100)

# Define a math evaluation field to compute mesh size based on distance
field_id_size = 2
gmsh.model.mesh.field.add("MathEval", field_id_size)
gmsh.model.mesh.field.setString(field_id_size, "F", "0.01 + 0.1 * F1")  # Min size 0.01, increases with distance

# Set the minimum field as the background mesh field
gmsh.model.mesh.field.add("Min", 3)
gmsh.model.mesh.field.setNumbers(3, "FieldsList", [field_id_size])
gmsh.model.mesh.field.setAsBackgroundMesh(3)

# Set global mesh parameters
gmsh.option.setNumber("Mesh.MeshSizeMin", 0.0003)  # Minimum element size
gmsh.option.setNumber("Mesh.MeshSizeMax", 0.01)   # Maximum element size
gmsh.option.setNumber("Mesh.Algorithm", 5)       # Delaunay for 3D meshing
gmsh.option.setNumber("Mesh.Format", 1)          # MSH format (version 2 ASCII)

# Generate the 3D mesh
gmsh.model.mesh.generate(3)

# Save the mesh to a file
gmsh.write("cylinder.msh")

# Optional: Launch Gmsh GUI to visualize the mesh
if '-nopopup' not in sys.argv:
    gmsh.fltk.run()

# Finalize Gmsh
gmsh.finalize()

Info    :  - Label 'Shapes/Living_hinge_1 v1' (3D)
Info    :  - Color (0.627451, 0.627451, 0.627451) (3D & Surfaces)
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Circle)
Info    : [ 20%] Meshing curve 4 (Line)
Info    : [ 20%] Meshing curve 5 (Circle)
Info    : [ 20%] Meshing curve 6 (Line)
Info    : [ 30%] Meshing curve 7 (Circle)
Info    : [ 30%] Meshing curve 8 (Line)
Info    : [ 30%] Meshing curve 9 (Line)
Info    : [ 40%] Meshing curve 10 (Line)
Info    : [ 40%] Meshing curve 11 (Line)
Info    : [ 40%] Meshing curve 12 (Line)
Info    : [ 50%] Meshing curve 13 (Circle)
Info    : [ 50%] Meshing curve 14 (Circle)
Info    : [ 50%] Meshing curve 15 (Line)
Info    : [ 60%] Meshing curve 16 (Line)
Info    : [ 60%] Meshing curve 17 (Circle)
Info    : [ 60%] Meshing curve 18 (Line)
Info    : [ 70%] Meshing curve 19 (Line)
Info    : [ 70%] Meshing curve 20 (Circle)
Info    : [ 70%] Meshing curve 2

File conversion of a .msh format into a more readable abaqus format : .inp 

In [3]:
import meshio

mesh = meshio.read("Living_hinge_1.msh")
meshio.write("Living_hinge_1.inp", mesh)

In [4]:
def extract_inp_sections_to_inp(input_path, output_path):
    
    with open(input_path, 'r') as f:
        lines = f.readlines()

    writing_nodes = False
    writing_elements = False
    nodes = []
    elements = []
    node_header = ""
    element_header = ""

    for line in lines:
        stripped = line.strip()
        upper = stripped.upper()

        if upper.startswith("*NODE"):
            writing_nodes = True
            writing_elements = False
            node_header = line.rstrip()
            continue
        elif upper.startswith("*ELEMENT"):
            writing_elements = True
            writing_nodes = False
            element_header = line.rstrip()
            continue
        elif upper.startswith("*ELSET"):  # stop reading at *ELSET or similar
            break

        if writing_nodes and stripped and not stripped.startswith("*"):
            nodes.append(line.rstrip())
        elif writing_elements and stripped and not stripped.startswith("*"):
            elements.append(line.rstrip())

    with open(output_path, 'w') as f:
        if node_header:
            f.write(f"{node_header}\n")
            for n in nodes:
                f.write(f"\t{n}\n")  # tab before each value line
        if element_header:
            f.write(f"{element_header}\n")
            for e in elements:
                f.write(f"\t{e}\n")  # tab before each value line

    print(f"Extracted .inp written to: {output_path}")

In [5]:
extract_inp_sections_to_inp('cylinder.inp', 'cylinder2.msh')

FileNotFoundError: [Errno 2] No such file or directory: 'cylinder.inp'